# 5. Context Managers

Directly from the notes because it's important: A very common software requirement is one you might call automatic wrap-up, which is to say that sometimes our programs perform operations where certain things need to be finalized or unwound when the operations have finished, whether the operations themselves succeeded or failed.

Suppose we wanted to read from a file. We can use a `try` to try and open the file, and `finally` to close it if it ever opened successfully. However, for more complicated problems this process can get tricky so we can instead use _context managers_ using a `with` statement.

```Python
with open(some_file_path, 'r', encoding='utf-8') as some_file:
    do_stuff(some_file)
```
Consider this example. The open function returns a file object stored into `some_file`, and this entire operation is encapsulated within the `with` statement.

As it turns out, file objects are _context managers_ so it has predefined some operations to perform when we exit this context, notably whether or not we should close the file based on whether opening it succeeded in the first place.

A common example is when we are unit testing and expect some code to fail in a particular way:
```Python
with self.assertRaises(SomeError):
    thing_that_triggers_some_error()
```
Unlike the file opening example we dont use `as` because we don't actually care about doing anything with the context manager in the body. 

### The `contextlib` module
This standard library module basically contains some context managers that might be useful to us, for example capturing standard input (particularly useful for testing):

In [6]:
import contextlib
import io
with contextlib.redirect_stdout(io.StringIO()) as output:
    print('hey there')
    # this print is captured by the context manager so doesn't print anything

In [7]:
# unless we ask for it
output.getvalue()

'hey there\n'

### Building context managers
Any object can be a context manager _if and only if_ it satisfies certain properties.
We can call these properties the context manager protocol, and anything that supports these operations can function as as context manager:
1. `__enter__(self)` - is called on the object as the `with` statement is entered. The return value of this function is stored into a variable if we use `as something`.
2. `__exit__(self, exc_type, exc_value, exc_traceback)` - is the opposite of enter. When the context manager exits, it passes in certain values to these parameters. If the `with` statement exits with no issues, all of these will get passed `None` as an argument, otherwise they will contain the type, value, and traceback of the exception respectively.

In [11]:
class SomeContextManager:
    def __init__(self, value):
        self.value = value
        print('constructed')
    
    def __enter__(self):
        print('entering')
        return self
    
    def __exit__(self, exc_type, exc_value, exc_traceback):
        if exc_type == None:
            print('successful exit')
        else:
            print(f'unsuccessful exit: {exc_type}')
            return True


In [12]:
with SomeContextManager('hello world') as x:
    print(x.value)

constructed
entering
hello world
successful exit


In [13]:
with SomeContextManager('goodbye world') as x:
    raise Exception

constructed
entering
unsuccessful exit: <class 'Exception'>


The reason an error does not come back in the second example, is because `__exit__` returned `True`, which basically tells the context manager "I have handled the issue so suppress it"